<a href="https://colab.research.google.com/github/GustavoFelips/DadosPagueBem/blob/main/Indices_PagueBem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importação da base

In [ ]:
import pandas as pd

#Criando o DataFrame através do CSV
df = pd.read_csv('data_pague_bem.csv')

#Retirando a Coluna Index que veio no CSV
df = df.drop(df.columns[0],axis=1)

#Verificando o DataFrame
df.head(3)


,codigo,identificador,nº parcela,nº documento,vencimento,pagamento,tempo para pagar,valor conta,valor pago,diferença,lead
0,1,34680,9,4728,2021-09-10,2021-09-11,-1,599.0,530.0,-69.0,3
1,1,34680,4,4728,2021-04-10,2021-04-13,-3,599.0,530.0,-69.0,3
2,1,34680,3,4728,2021-03-10,2021-03-11,-1,599.0,530.0,-69.0,3


# Índice De Pagamento

##Função Índice Pagamento

In [ ]:
#Função que calcula a nota do índice a partir da média do tempo de pagamento
def calcular_indice_pagamento(media):
  if media < -15:
    return 0
  elif -15 <= media < -10:
    return 2.5
  elif -10 <= media < -5:
    return 5.0
  elif -5 <= media < -2:
    return 7.0
  elif -2 <= media < 0:
    return 8
  elif media >= 0:
    return 10

##Índice de todos os clientes



In [ ]:

#Tira as médias do tempo de pagamento por cliente e retorna suas devidas notas
def tabela_indice_pagamento(df):
  if 'identificador' in df.columns and 'tempo para pagar' in df.columns:

    #Calcula a média de tempo de pagamento por cliente
    medias_por_cliente = df.groupby('identificador')['tempo para pagar'].mean().reset_index()

    #Calcula para cada média uma nota
    medias_por_cliente['i.pag'] = medias_por_cliente['tempo para pagar'].apply(calcular_indice_pagamento)

    #Modifica nome da coluna
    medias_por_cliente.rename(columns={'tempo para pagar' : 'media_tempo_pagamento'},inplace=True)

    return medias_por_cliente

  else:

    print("As colunas 'identificador' ou 'tempo para pagar' não foram encontradas no arquivo CSV.")

#Chamada da função
tabela_indice_pagamento(df)



,identificador,media_tempo_pagamento,i.pag
0,16075,-9.333333,5.0
1,16077,6.666667,10.0
2,16078,0.333333,10.0
3,16080,2.222222,10.0
4,16081,-117.888889,0.0
...,...,...,...
1102,40438,0.888889,10.0
1103,42085,6.555556,10.0
1104,42228,-7.777778,5.0
1105,43881,4.000000,10.0


##Índice por Cliente



In [ ]:
def indice_pagamento_por_cliente(cliente_id):
    linhas_cliente = df[df['identificador']==cliente_id]
    media_tempo_pagamento = linhas_cliente['tempo para pagar'].mean()
    return {'media_tempo_pagamento':media_tempo_pagamento,'indice_pagamento':calcular_indice_pagamento(media_tempo_pagamento)}

#Exemplo
indice_pagamento_por_cliente(16075)


{'media_tempo_pagamento': -9.333333333333334, 'indice_pagamento': 5.0}

# Índice de Regularidade

##Funçao Índice Regularidade

In [ ]:
def calcular_indice_regularidade(desvio_padrao):
    # Atribuir a nota com base no desvio padrão que já foi calculado
    if desvio_padrao > 30:
        return 0
    elif 20 < desvio_padrao <= 30:
        return 2.5
    elif 10 < desvio_padrao <= 20:
        return 5
    elif 5 < desvio_padrao <= 10:
        return 7
    elif 3 < desvio_padrao <= 5:
        return 8
    elif 0 <= desvio_padrao <= 3:
        return 10


##Índice de todos os clientes

In [ ]:
def tabela_indice_regularidade (df) :
    df_indice = df.groupby('identificador')['tempo para pagar'].agg(
    desvio_padrao='std',  # Calcula o desvio padrão do tempo de pagamento por cliente
    quantidade_pagamentos='count').reset_index() # Quantidade de pagamentos por cliente (pode ser útil)

    # Aplicar a função calcular_indice_regularidade
    df_indice['i.reg'] = df_indice['desvio_padrao'].apply(calcular_indice_regularidade)

    return df_indice

# Exibir as primeiras linhas do DataFrame
tabela_indice_regularidade(df)

,identificador,desvio_padrao,quantidade_pagamentos,i.reg
0,16075,24.909837,9,2.5
1,16077,10.037430,9,5.0
2,16078,2.828427,9,10.0
3,16080,2.048034,9,10.0
4,16081,83.257799,9,0.0
...,...,...,...,...
1102,40438,2.803767,9,10.0
1103,42085,4.362084,9,8.0
1104,42228,11.278051,9,5.0
1105,43881,23.113849,9,2.5


##Índice por Cliente

In [ ]:
def indice_regularidade_por_cliente(cliente_id):
    # Filtrar o DataFrame original pelo cliente_id
    df_cliente = df[df['identificador'] == cliente_id]

    # Calcular o desvio padrão do 'tempo para pagar'
    desvio_padrao = df_cliente['tempo para pagar'].std()

    # Chamar a função que calcula o índice de regularidade
    return {'desvio_padrao':desvio_padrao,'indice_regularidade':calcular_indice_regularidade(desvio_padrao)}

# Exemplo de uso
indice_regularidade_por_cliente(16075)


{'desvio_padrao': 24.9098374141623, 'indice_regularidade': 2.5}

#índice de Interação

##Função Índice Interação

In [ ]:
#Função para calcular o índice de Iteração

def calcular_indice_interacao(lead):
    '''
        Valor 0: Frio.
        Valor 1: Aquecendo.
        Valor 2: Quente.
        Valor 3: Convertido.
                             '''
    nota = lead * 3.333

    return round(nota,2)

calcular_indice_interacao(8)


26.66

##Indice de todos os Clientes

In [ ]:
#Retorna uma tabela com o índice de interação de cada cliente
def tabela_indice_interacao(df):
  if 'lead' in df.columns:
    #Média do lead dos clientes
    medias_lead_por_cliente = df.groupby('identificador')['lead'].mean().reset_index()

    #Calcula o índice de interação
    medias_lead_por_cliente['i.int'] = medias_lead_por_cliente['lead'].apply(calcular_indice_interacao)

    #Renomeia o nome da coluna
    medias_lead_por_cliente.rename(columns={'lead' : 'media_lead'},inplace=True)

    return medias_lead_por_cliente

tabela_indice_interacao(df).head()

,identificador,media_lead,i.int
0,16075,2.888889,9.63
1,16077,3.000000,10.00
2,16078,3.000000,10.00
3,16080,2.888889,9.63
4,16081,0.222222,0.74


##Índice por Cliente

In [ ]:
def indice_regularidade_por_cliente(cliente_id):
    # Filtrar o DataFrame original pelo cliente_id
    df_cliente = df[df['identificador'] == cliente_id]

    # Calcular a media do lead
    media = df_cliente['lead'].mean()

    # Chamar a função que calcula o índice de interação
    return {'media_lean':media,'indice_interacao':calcular_indice_interacao(media)}

# Exemplo de uso
indice_regularidade_por_cliente(16077)

{'media_lean': 3.0, 'indice_interacao': 10.0}

#Índice de Reputação

##Tabela Completa

In [ ]:
df_cheia = pd.merge(tabela_indice_pagamento(df), tabela_indice_regularidade(df), on='identificador', how='outer').merge(tabela_indice_interacao(df), on='identificador', how='outer')
df_cheia = df_cheia[['identificador','quantidade_pagamentos','media_tempo_pagamento','media_lead','desvio_padrao','i.pag','i.reg','i.int']]
df_cheia.head(3)

,identificador,quantidade_pagamentos,media_tempo_pagamento,media_lead,desvio_padrao,i.pag,i.reg,i.int
0,16075,9,-9.333333,2.888889,24.909837,5.0,2.5,9.63
1,16077,9,6.666667,3.000000,10.037430,10.0,5.0,10.00
2,16078,9,0.333333,3.000000,2.828427,10.0,10.0,10.00


#Função do Índice





In [ ]:
def calcular_indice_reputacao(row):
    # Extrair os valores dos índices da linha
    I_Pag = row['i.pag']
    I_Reg = row['i.reg']
    I_Int = row['i.int']

    # Calcular o I.Rep com base na fórmula
    I_Rep = ((I_Pag * 5) + (I_Reg * 3) + (I_Int * 2)) / 10
    return I_Rep

##Teste da função


In [ ]:
# Aplicar a função em todas as linhas do DataFrame
df_cheia = pd.merge(tabela_indice_pagamento(df), tabela_indice_regularidade(df), on='identificador', how='outer').merge(tabela_indice_interacao(df), on='identificador', how='outer')

df_cheia['i.rep'] = df_cheia.apply(calcular_indice_reputacao, axis=1)

#Criando dataframe exclusivo do índice
df_rep = df_cheia[['identificador', 'i.rep']]

# Visualizar as primeiras linhas do DataFrame
df_rep.head(10)

,identificador,i.rep
0,16075,5.176
1,16077,8.500
2,16078,10.000
3,16080,9.926
4,16081,0.148
5,16082,1.334
6,16084,2.676
7,16086,4.750
8,16088,1.778
9,16092,0.740


##Função do índice


In [ ]:
# Filtrar o DataFrame para o cliente específico (exemplo: cliente com 'codigo' 123)
cliente_especifico = df_cheia.loc[df_cheia['identificador'] == 16075]

# Aplicar a função de cálculo do I.Rep apenas para esse cliente
cliente_especifico.loc[:, 'i.rep'] = cliente_especifico.apply(calcular_indice_reputacao, axis=1)

# Visualizar o resultado
cliente_especifico.head()

,identificador,media_tempo_pagamento,i.pag,desvio_padrao,quantidade_pagamentos,i.reg,media_lead,i.int,i.rep
0,16075,-9.333333,5.0,24.909837,9,2.5,2.888889,9.63,5.176


#Tabela Completa

In [ ]:
df_completo = pd.merge(tabela_indice_pagamento(df), tabela_indice_regularidade(df), on='identificador', how='outer').merge(tabela_indice_interacao(df), on='identificador', how='outer')
df_completo = df_completo[['identificador','quantidade_pagamentos','media_tempo_pagamento','media_lead','desvio_padrao','i.pag','i.reg','i.int']]
df_completo = pd.merge(df_completo, df_rep, on='identificador', how='left')

df_completo.head(10)

,identificador,quantidade_pagamentos,media_tempo_pagamento,media_lead,desvio_padrao,i.pag,i.reg,i.int,i.rep
0,16075,9,-9.333333,2.888889,24.909837,5.0,2.5,9.63,5.176
1,16077,9,6.666667,3.000000,10.037430,10.0,5.0,10.00,8.500
2,16078,9,0.333333,3.000000,2.828427,10.0,10.0,10.00,10.000
3,16080,9,2.222222,2.888889,2.048034,10.0,10.0,9.63,9.926
4,16081,9,-117.888889,0.222222,83.257799,0.0,0.0,0.74,0.148
5,16082,9,-129.444444,2.000000,93.477419,0.0,0.0,6.67,1.334
6,16084,9,-31.333333,2.888889,26.650516,0.0,2.5,9.63,2.676
7,16086,9,-11.333333,3.000000,19.805302,2.5,5.0,10.00,4.750
8,16088,9,-63.333333,2.666667,38.910153,0.0,0.0,8.89,1.778
9,16092,9,-102.111111,1.111111,51.116153,0.0,0.0,3.70,0.740
